# Portfolio Simulation
This notebook contains an analysis on different strategies for a portfolio. It performs a Montecarlo simulation with Bootstrap to figure out the expected return in various scenarios, expected DrawDown, volatility, and 5% percentile worst scenarios

In [4]:
import json
import csv
import pandas as pd
import numpy as np
import os

In [5]:
# write a function to trasform the json data in csv file given a json file that is an array of objects with a date and a value
def json_to_csv(json_file, csv_file):
    with open(json_file, 'r') as jf:
        data = json.load(jf)

    if not data:
        print("No data found in the JSON file.")
        return

    # Assuming all objects have the same keys
    keys = data[0].keys()

    with open(csv_file, 'w', newline='') as cf:
        writer = csv.DictWriter(cf, fieldnames=keys)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

# given a directory, convert all json files in that directory to csv files
def convert_all_json_to_csv(directory_json, directory_csv):
    for filename in os.listdir(directory_json):
        if filename.endswith('.json'):
            json_file = os.path.join(directory_json, filename)
            csv_file = os.path.join(directory_csv, filename.replace('.json', '.csv'))
            json_to_csv(json_file, csv_file)
            
convert_all_json_to_csv("data/json", "data/csv")

In [3]:
def calculate(file, freq="monthly", verbose=False):

    # 1️⃣ Carica i dati
    df = pd.read_csv(file, parse_dates=["date"])
    df = df.sort_values("date").set_index("date")

    # 2️⃣ Calcola rendimenti logaritmici
    df["log_ret"] = np.log(df["value"] / df["value"].shift(1))
    # oppure rendimenti percentuali:
    # df["pct_ret"] = df["value"].pct_change()

    # 3️⃣ Filtra eventuali NaN iniziali
    df = df.dropna(subset=["log_ret"])

    # 4️⃣ Calcola statistiche
    mean_period = df["log_ret"].mean()
    std_period = df["log_ret"].std()  # ddof=0 per vol historical

    # 5️⃣ Annualizzazione: se hai dati mensili (12), o giornalieri (252 trading days)
    if freq == "daily":
        periods = 252
    elif freq == "monthly":
        periods = 12

    annual_mean = mean_period * periods
    annual_vol = std_period * np.sqrt(periods)
    if verbose:
        print(f"Rendimento atteso annuo: {annual_mean*100:.2f}%")
        print(f"Volatilità annuale: {annual_vol*100:.2f}%")
    return df

# Example usage
def print_all(directory):
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            print(filename)
            calculate(os.path.join(directory, filename), verbose=True)
            print("\n")

print_all("data/csv")  # Replace with your actual CSV directory path


bond-7-10.csv
Rendimento atteso annuo: 3.39%
Volatilità annuale: 5.69%


cash.csv
Rendimento atteso annuo: 0.98%
Volatilità annuale: 0.48%


gold.csv
Rendimento atteso annuo: 6.18%
Volatilità annuale: 17.14%


msci-world.csv
Rendimento atteso annuo: 9.82%
Volatilità annuale: 14.95%


msci-world-quality.csv
Rendimento atteso annuo: 7.47%
Volatilità annuale: 14.14%


msci-world-momentum.csv
Rendimento atteso annuo: 10.63%
Volatilità annuale: 15.40%


msci-world-min-vol.csv
Rendimento atteso annuo: 7.81%
Volatilità annuale: 11.83%


msci-world-value.csv
Rendimento atteso annuo: 8.19%
Volatilità annuale: 16.24%


msci-world-small-cap.csv
Rendimento atteso annuo: 7.31%
Volatilità annuale: 16.93%




In [2]:
# given two assets, compute the correlation between them
def compute_correlation(asset1, asset2):
    df1 = calculate(asset1)
    df2 = calculate(asset2)

    # Trova l’intersezione delle date comuni
    common_index = df1.index.intersection(df2.index)

    # Allinea i dataframe solo alle date comuni
    aligned1 = df1.loc[common_index]
    aligned2 = df2.loc[common_index]

    # Calcola la matrice di correlazione
    correlation = aligned1['log_ret'].corr(aligned2['log_ret'])

    print("Correlation:", correlation)

def correlation_matrix(folder):
    files = [f for f in os.listdir(folder) if f.endswith('.csv')]
    data = {}

    for file in files:
        df = calculate(os.path.join(folder, file))
        data[file] = df['log_ret']

    # Crea un DataFrame con i rendimenti logaritmici
    returns_df = pd.DataFrame(data)

    # Calcola la matrice di correlazione
    correlation_matrix = returns_df.corr()

    return correlation_matrix

def build_single_dataframe(folder):
    files = [f for f in os.listdir(folder) if f.endswith('.csv')]
    data = {}

    for file in files:
        df = calculate(os.path.join(folder, file))
        data[file] = df['log_ret']

    # Crea un DataFrame con i rendimenti logaritmici
    returns_df = pd.DataFrame(data)

    return returns_df

correlation_matrix("data/csv/")
build_single_dataframe("data/csv/")

NameError: name 'calculate' is not defined

In [2]:
def montecarlo_bootstrap_monthly(returns_df, n_simulations=1000, horizon=60):
    """
    returns_df: DataFrame con rendimenti mensili (righe = mesi, colonne = asset)
    n_simulations: numero di simulazioni Monte Carlo
    horizon: orizzonte in mesi da simulare (es. 60 mesi = 5 anni)
    """
    asset_names = returns_df.columns

    # Matrice di correlazione e decomposizione di Cholesky
    correlation_matrix = returns_df.corr().values
    chol = np.linalg.cholesky(correlation_matrix)

    simulations = []

    for _ in range(n_simulations):
        # 1. Bootstrap dei rendimenti: righe con rimpiazzo (random pick)
        sampled = returns_df.sample(n=horizon, replace=True).values.T  # shape (n_assets, horizon)

        # 2. Applica correlazione
        correlated = chol @ sampled  # shape (n_assets, horizon)

        # 3. Ritorna in forma di DataFrame
        sim_df = pd.DataFrame(correlated.T, columns=asset_names)
        simulations.append(sim_df)

    return simulations  # lista di DataFrame (uno per simulazione)

def load_log_return_series_from_csv(filepath, asset_name):
    df = pd.read_csv(filepath, parse_dates=["date"])
    df = df.sort_values("date").set_index("date")

    # filter for dates after 1998-12-01
    df = df[df.index > "1998-12-01"]

    # Calcola log-returns: ln(Pt / Pt-1)
    df["log_return"] = np.log(df["value"] / df["value"].shift(1))
    df = df[["log_return"]].dropna()
    df.columns = [asset_name]

    return df


def create_simulations(all_dfs, store = False):
    returns_df = pd.concat([df["df"] for df in all_dfs], axis=1).dropna()

    simulations = {}

    for horizon in [60, 120, 180, 240]:
        print(f"Simulazioni per orizzonte di {horizon} mesi:")
        sims = montecarlo_bootstrap_monthly(returns_df, n_simulations=10000, horizon=horizon)
        simulations[horizon] = sims
        print(f"Numero di simulazioni: {len(sims)}\n")
        if store:
            # Salva le simulazioni in un file CSV
            output_dir = f"data/simulations/horizon_{horizon}"
            os.makedirs(output_dir, exist_ok=True)
            for i, sim in enumerate(sims):
                sim.to_csv(os.path.join(output_dir, f"simulation_{i+1}.csv"), index=True)

    return simulations


all_dfs = [{"name": f.replace(".csv",""), "df": load_log_return_series_from_csv(os.path.join("data/csv", f), f.replace(".csv",""))} for f in os.listdir("data/csv") if f.endswith('.csv')]

montecarlo_simulations = create_simulations(all_dfs)


Simulazioni per orizzonte di 60 mesi:
Numero di simulazioni: 10000

Simulazioni per orizzonte di 120 mesi:
Numero di simulazioni: 10000

Simulazioni per orizzonte di 180 mesi:
Numero di simulazioni: 10000

Simulazioni per orizzonte di 240 mesi:
Numero di simulazioni: 10000



In [5]:
# create a copy of the simulations for portfolio analysis
portfolios_weights = [
    {"name": "60-40", "composition": [("msci-world", 0.6), ("bond-7-10", 0.4)]},
    {"name": "All-Weather World", "composition": [("msci-world", 0.5), ("bond-7-10", 0.3), ("gold", 0.1), ("cash", 0.1)]},
    {"name": "Factors", "composition": [("msci-world-momentum", 0.45),("msci-world-min-vol", 0.25), ("bond-7-10", 0.2), ("gold", 0.1)]},
    {"name": "Factors + Cash", "composition": [("msci-world-momentum", 0.35),("msci-world-min-vol", 0.25), ("bond-7-10", 0.2), ("gold", 0.1), ("cash", 0.1)]}
]

portfolios_weights = [
    {"name": "60-40", "composition": [("msci-world", 0.6), ("bond-7-10", 0.4)]},
    {"name": "All-Weather World", "composition": [("msci-world", 0.5), ("bond-7-10", 0.3), ("gold", 0.1), ("cash", 0.1)]},
    {"name": "Factors + Cash", "composition": [("msci-world-momentum", 0.35),("msci-world-min-vol", 0.25), ("bond-7-10", 0.2), ("gold", 0.1), ("cash", 0.1)]}
]
portfolio_simulations = {horizon: [sim.copy() for sim in montecarlo_simulations[horizon]] for horizon in montecarlo_simulations}
for horizon in montecarlo_simulations:
    # build different portfolios, e.g., equal weight, market cap weight, etc.
    for i, sim in enumerate(montecarlo_simulations[horizon]):
        for weight in portfolios_weights:
            # create a portfolio based on the weights
            portfolio_sim = sum(sim[asset[0]] * weight['composition'][j][1] for j, asset in enumerate(weight['composition']))
            # set the name of the portfolio
            portfolio_sim.name = f"{weight['name']}_Portfolio_{i+1}_Horizon_{horizon}"
            portfolio_simulations[horizon][i][weight['name']] = portfolio_sim

        # # Example: equal weight portfolio
        # portfolio_sim = sim.mean(axis=1)
        # # set the name of the portfolio
        # portfolio_sim.name = f"Portfolio_{i+1}_Horizon_{horizon}"
        # portfolio_simulations[horizon][i]['equal-weighted'] = portfolio_sim

: 

In [30]:
# for each simulation, calculate the mean and standard deviation of the returns
def calculate_simulation_stats(simulations):
    stats = {}
    for horizon, sim_list in simulations.items():
        mean_returns = []
        std_returns = []
        for sim in sim_list:
            mean_returns.append(sim.mean())
            std_returns.append(sim.std())
        stats[horizon] = {
            "mean": pd.DataFrame(mean_returns).mean(),
            "std": pd.DataFrame(std_returns).mean()
        }
    return stats

simulation_stats = calculate_simulation_stats(montecarlo_simulations)
simulation_stats

{60: {'mean': bond-7-10               0.002824
  cash                    0.000934
  gold                    0.008563
  msci-world              0.005633
  msci-world-quality      0.007213
  msci-world-momentum     0.009395
  msci-world-min-vol      0.010912
  msci-world-value        0.003645
  msci-world-small-cap    0.006130
  dtype: float64,
  'std': bond-7-10               0.016313
  cash                    0.001863
  gold                    0.044704
  msci-world              0.039976
  msci-world-quality      0.045758
  msci-world-momentum     0.056268
  msci-world-min-vol      0.054791
  msci-world-value        0.040747
  msci-world-small-cap    0.048271
  dtype: float64},
 120: {'mean': bond-7-10               0.002830
  cash                    0.000934
  gold                    0.008439
  msci-world              0.005661
  msci-world-quality      0.007238
  msci-world-momentum     0.009427
  msci-world-min-vol      0.010920
  msci-world-value        0.003685
  msci-world-small-ca

In [4]:
# for each simulation calculate the cumulative returns
def calculate_cumulative_returns(simulations):
    cumulative_returns = {}
    for horizon, sim_list in simulations.items():
        cum_returns = []
        for sim in sim_list:
            cum_return = (1 + sim).cumprod() - 1  # Calcola i rendimenti cumulativi
            cum_returns.append(cum_return)
        cumulative_returns[horizon] = cum_returns
    return cumulative_returns

def analyze_simulations(simulations):
    final_returns_list = []
    variances_list = []

    for sim in simulations:
        # Calcolo del rendimento cumulativo per ogni asset (log → somma, altrimenti moltiplica)
        cumulative = (1 + sim).sum() - 1  # se usi pct_change; usa .sum() se hai log-returns
        variance = sim.std()  # varianza su tutti i mesi

        final_returns_list.append(cumulative)
        variances_list.append(variance)

    # Combina i risultati in DataFrame
    final_returns_df = pd.DataFrame(final_returns_list)
    variances_df = pd.DataFrame(variances_list)

    # Rendimento medio finale e varianza media su tutte le simulazioni
    mean_final_return = final_returns_df.mean()
    mean_variance = variances_df.mean()

    return mean_final_return, mean_variance

def max_drawdown(rets_df, initial_value=100):
    # 1. Calcola indice di ricchezza cumulato (wealth index)
    wealth = (1 + rets_df).cumprod() * initial_value

    # 2. Calcola picco cumulativo (running max)
    running_max = wealth.cummax()

    # 3. Calcola drawdown in ogni istante
    drawdown = wealth / running_max - 1

    # 4. Massimo drawdown (più negativo) per asset
    mdd = drawdown.min()

    return mdd


def summarize_simulations(simulations, months, use_log_returns=False):
    final_returns = []
    variances = []
    max_drawdowns = []

    for sim in simulations:
        if use_log_returns:
            cumulative_return = np.exp(sim.sum()) - 1
        else:
            cumulative_return = (1 + sim).prod() - 1

        variance = sim.std()

        ## annualizza il rendimento cumulativo e la varianza
        cumulative_return_annualized = (1 + cumulative_return) ** (12 / months) - 1
        variance_annualized = variance
        mdd = max_drawdown(sim)

        final_returns.append(cumulative_return_annualized)
        variances.append(variance_annualized)
        max_drawdowns.append(mdd)

    # Costruisci i DataFrame
    final_returns_df = pd.DataFrame(final_returns)    # ogni riga = 1 simulazione
    variances_df = pd.DataFrame(variances)
    max_drawdowns_df = pd.DataFrame(max_drawdowns)

    return final_returns_df, variances_df, max_drawdowns_df

def horizon_summary(montecarlo_simulations, horizon, use_log_returns=False):
    ret_df, var_df, mdd_df = summarize_simulations(montecarlo_simulations[horizon], horizon, use_log_returns=use_log_returns)

    # Calcola il ritorno finale sapendo che red_df contiene il ritorno annualizzato, quindi elevato a 12/horizon
    final_df = (1 + ret_df) ** (horizon / 12) - 1


    # Analisi per asset
    summary = pd.DataFrame({
        "mean_return": ret_df.mean(),
        "median_return": ret_df.median(),
        "5th_percentile_return": ret_df.quantile(0.05),
        "horizon_mean_return": final_df.mean(),
        "horizon_median_return": final_df.median(),
        "horizon_5th_percentile_return": final_df.quantile(0.05),
        "mean_variance": var_df.mean(),
        "median_variance": var_df.median(),
        "max_dd": mdd_df.quantile(0.05),
    })

    return summary

import matplotlib.pyplot as plt

def save_describe_as_image(df, title, filename="describe_output.png"):
    desc = df

    fig, ax = plt.subplots(figsize=(12, 4))
    ax.axis('off')  # nasconde gli assi
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    table = ax.table(
        cellText=desc.values,
        colLabels=desc.columns,
        rowLabels=desc.index,
        loc='center',
        cellLoc='center'
    )
    table.scale(1, 1.5)  # opzionale: regola dimensione
    plt.tight_layout()
    plt.savefig(filename, dpi=300)
    plt.close()


# for months in [60, 120, 180, 240]:
#     summ = horizon_summary(montecarlo_simulations, months, use_log_returns=True)
#     save_describe_as_image(summ, f"Summary for {months} months", filename=f"summary_{months}_months.png")
#     print(f"Summary for {months} months:\n", summ)
months = 240
summ = horizon_summary(portfolio_simulations, months, use_log_returns= False)

save_describe_as_image(summ, f"Summary for {months} months", filename=f"summary_{months}_months.png")
summ

,mean_return,median_return,5th_percentile_return,horizon_mean_return,horizon_median_return,horizon_5th_percentile_return,mean_variance,median_variance,max_dd
bond-7-10,0.032952,0.032907,0.010990,0.973860,0.910846,0.244331,0.016356,0.016341,-0.226663
cash,0.011215,0.011210,0.008834,0.250394,0.249765,0.192336,0.001867,0.001864,-0.010516
gold,0.094192,0.093399,0.034028,6.571543,4.964534,0.952736,0.044870,0.044831,-0.493634
msci-world,0.060407,0.060262,0.006470,2.883254,2.223047,0.137676,0.040165,0.040197,-0.538104
msci-world-quality,0.077486,0.077336,0.014957,4.655364,3.436324,0.345704,0.045970,0.046003,-0.567993
msci-world-momentum,0.099062,0.098106,0.020798,8.508434,5.499552,0.509382,0.056510,0.056531,-0.645018
msci-world-min-vol,0.119984,0.119326,0.042159,12.552584,8.530901,1.283901,0.055027,0.055041,-0.585912
msci-world-value,0.035158,0.034888,-0.018799,1.420764,0.985504,-0.315832,0.040989,0.041000,-0.631628
msci-world-small-cap,0.062093,0.061825,-0.003545,3.370898,2.319391,-0.068557,0.048541,0.048562,-0.640895
60-40,0.051884,0.052032,0.017473,1.963877,1.757887,0.414034,0.025671,0.025672,-0.341258


In [58]:
def max_drawdown(rets_df, initial_value=100):
    # 1. Calcola indice di ricchezza cumulato (wealth index)
    wealth = (1 + rets_df).cumprod() * initial_value
    print(wealth)

    # 2. Calcola picco cumulativo (running max)
    running_max = wealth.cummax()

    # 3. Calcola drawdown in ogni istante
    drawdown = wealth / running_max - 1

    # 4. Massimo drawdown (più negativo) per asset
    mdd = drawdown.min()

    return mdd

test = [0.01, 0.02, -0.03, 0.04, -0.05, -0.02, 0.03, -0.05, -0.05, 0.1,0.06, -0.1]
test_df = pd.DataFrame(test, columns=["log_ret"])
mdd = max_drawdown(test_df)
print("Max Drawdown:", mdd)

       log_ret
0   101.000000
1   103.020000
2    99.929400
3   103.926576
4    98.730247
5    96.755642
6    99.658312
7    94.675396
8    89.941626
9    98.935789
10  104.871936
11   94.384742
Max Drawdown: log_ret   -0.134566
dtype: float64


In [66]:
cumulative_returns[60][0].tail()

,bond-7-10,cash,gold,msci-world,msci-world-quality,msci-world-momentum,msci-world-min-vol,msci-world-value,msci-world-small-cap
55,0.302479,0.072307,1.300521,0.788834,1.092166,1.428619,2.084894,0.485656,1.056437
56,0.296903,0.071661,1.325098,0.840340,1.161571,1.530370,2.211416,0.531517,1.126781
57,0.312551,0.072245,1.367370,0.743000,1.030358,1.361050,2.047011,0.434176,0.984354
58,0.343648,0.075009,1.273690,0.717692,0.992200,1.345921,2.050459,0.404143,0.920672
59,0.355828,0.079356,1.371618,0.729230,1.015323,1.390997,2.112399,0.402479,0.945573
